# Read alignement methods

In [106]:
# Download FASTA file excerpt of human chromosome 1
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2022-12-09 19:48:13--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 143.204.222.132, 143.204.222.99, 143.204.222.215, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|143.204.222.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta.1’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  77.3KB/s    in 6.7s    

2022-12-09 19:48:24 (119 KB/s) - ‘chr1.GRCh38.excerpt.fasta.1’ saved [810105/810105]



In [107]:
# Function provided by course
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
genome = readGenome('chr1.GRCh38.excerpt.fasta')
genome[:100]

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA'

In [108]:
len(genome)

800000

<h3 style="border-bottom: 1px solid">Naive VS Boyer-Moore method</h3>
**Objective** : Build third version of naive exact matching and Boyer-Moore algorithms that additionally count and return 
 1. the number of character comparisons performed and 
 2. the number of alignments tried. Roughly speaking, these measure how much work the two different algorithms are doing
 3. list of offsets of occurences

In [109]:
""" Do naive exact matching """
def naive_with_counts(p, t):
    occurrences = []
    numbCharComp = 0
    numbAlignments = 0
    for i in range(len(t) - len(p) + 1):
        match = True
        numbAlignments += 1
        for j in range(len(p)):
            numbCharComp += 1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences, numbAlignments, numbCharComp

In [110]:
#function to get complementary strand of sequence (imported from course module)
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

<div class="alert alert-block alert-success" style="text-align:center">Naive matching Test 1</div>

In [111]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


<div class="alert alert-block alert-success" style="text-align:center">Naive matching Test 2</div>

In [112]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)


[0, 19] 20 35


In [113]:
from bm_preproc import BoyerMoore

In [114]:
""" Do Boyer-Moore matching """
def boyer_moore_with_counts(p, p_bm, t):
    i = 0
    occurrences = []
    numbCharComp = 0
    numbAlignments = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        numbAlignments += 1
        for j in range(len(p)-1, -1, -1):
            numbCharComp += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences,numbAlignments,numbCharComp 

<div class="alert alert-block alert-success" style="text-align:center">Boyer-Moore matching Test 1</div>

In [115]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


<div class="alert alert-block alert-success" style="text-align:center">Boyer-Moore matching Test 2</div>

In [116]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


### Question 1
How many alignments does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [117]:
p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
_, num_alignments, num_character_comparisons = naive_with_counts(p, genome)
print('The naive exact matching algorithm tries %d alignments' % (num_alignments))

The naive exact matching algorithm tries 799954 alignments


### Question 2
How many character comparisons does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [118]:
print('The naive exact matching algorithm tries %d character comparisons' % (num_character_comparisons))

The naive exact matching algorithm tries 984143 character comparisons


### Question 3
How many alignments does Boyer-Moore try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [119]:
p_bm = BoyerMoore(p, alphabet='ACGT')
_, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, genome)
print('The Boyer-Moore algorithm tries %d alignments' % (num_alignments))

The Boyer-Moore algorithm tries 127974 alignments


In [120]:
print('The Boyer-Moore algorithm tries %d character comparisons' % (num_character_comparisons))

The Boyer-Moore algorithm tries 165191 character comparisons


<h3 style="border-bottom: 1px solid">k-mer indexing method</h3>
<ol>
    <li>Build indexing table for genome with k-mers</li>
    <li>Create P partitions with pigeonhole principle</li>
    <li>Search for exact match for one partition</li>
    <li>Verify match</li>
</ol>

In [121]:
import bisect

In [122]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

**Problem** : Build function that, given a length-24 pattern P and given an Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.


In [123]:
''' function: divide a string into substrings of length k '''
def splitArray(text, k):
    i = 0
    result = []
    offsets = []
    while i < len(text):
        offsets.append(i)
        result.append(text[i:min(i+k,len(text))])
        i += k
    return result,offsets 

In [124]:
ten_as = 'AAAAAAAAAA'
t = ten_as + 'CTGT' + ten_as + 'CTTT' + ten_as + 'CGGG' + ten_as
result,offsets = splitArray(t, 4)
print(result)
print(offsets)
print(len(t))

['AAAA', 'AAAA', 'AACT', 'GTAA', 'AAAA', 'AAAA', 'CTTT', 'AAAA', 'AAAA', 'AACG', 'GGAA', 'AAAA', 'AAAA']
[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48]
52


In [125]:
''' function: use k-mer index data to find matches '''
def k_mer_match_2mm(p, t, index):
    k = index.k
    occurrences = []
    totalHits = 0
    # split p into substrings of length k
    partitions, partitionOffsets = splitArray(p, k)
    
    #for each partition of p
    for idx, target in enumerate(partitions):
        indexHitsForPartition = index.query(target)
            
        #if match found
        if len(indexHitsForPartition)>0:
            '''Count total index hits'''
            totalHits += len(indexHitsForPartition)
            
            leftOffsetTarget = partitionOffsets[idx]
            rightOffsetTarget = partitionOffsets[idx]+k
            partitionAfter = len(p) - rightOffsetTarget
            
            #for every index hit
            for i in indexHitsForPartition:
                
                # if hit is not already added
                if occurrences.count(i-k)==0:
                    '''Count missmatch on all of p'''
                    mismatch = 0 

                    # if match is at the end of t
                    if (i+partitionAfter) > len(t):
                        mismatch = (i+partitionAfter) - len(t)

                    # if match is at the begining of t
                    if leftOffsetTarget > i:
                        mismatch += leftOffsetTarget - i
                    
                    '''Count mismatch on sequence prefix to match partition'''
                    nt = 0
                    while nt < partitionAfter and mismatch<=2:
                        #if mismatch
                        if t[i+k+nt] != p[rightOffsetTarget+nt]:
                            mismatch += 1
                        nt += 1
                        
                    '''Count mismatch on sequence suffix to match partition'''
                    nt = 0
                    while nt < leftOffsetTarget and mismatch<=2:
                        #if mismatch
                        if t[i-leftOffsetTarget+nt] != p[nt]:
                            mismatch += 1
                        nt += 1    

                    #if #mismatch <= 2 on all of P append hit
                    if mismatch<=2:
                        occurrences.append(i-leftOffsetTarget)
                        occurrences.sort()
        
    return occurrences, totalHits

<div class="alert alert-block alert-success" style="text-align:center">k-mer with substring matching Test 1</div>

In [126]:
p = 'CTGT'
ten_as = 'AAAAAAAAAA'
t = ten_as + 'CTGT' + ten_as + 'CTTT' + ten_as + 'CGGG' + ten_as
index = Index(t, 2)
occurences,_=k_mer_match_2mm(p, t, index)
print(occurences)

[10, 24]


<div class="alert alert-block alert-success" style="text-align:center">k-mer with substring matching Test 2</div>

In [127]:
# Load phix_genome
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2022-12-09 19:48:28--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 143.204.222.132, 143.204.222.99, 143.204.222.215, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|143.204.222.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: ‘phix.fa.1’

phix.fa.1           100%[===================>]   5.40K  --.-KB/s    in 0.003s  

2022-12-09 19:48:29 (1.66 MB/s) - ‘phix.fa.1’ saved [5528/5528]



In [128]:
phix_genome = readGenome('phix.fa')
index = Index(phix_genome, 2)
occurrences,_ = k_mer_match_2mm("GATTACA", phix_genome, index)
print('offset of leftmost occurrence: %d' % min(occurrences))

offset of leftmost occurrence: 10


### Question 4
How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

In [129]:
p="GGCGCGGTGGCTCACGCCTGTAAT"
index = Index(genome, 8)
occurrences,totalHits = k_mer_match_2mm(p, genome, index)
print( p + ' occurs %d times in the excerpt of human chromosome 1' % len(occurrences))

GGCGCGGTGGCTCACGCCTGTAAT occurs 35 times in the excerpt of human chromosome 1


<div class="alert alert-block alert-success" style="text-align:center">Verify matches </div>

In [130]:
mismatchAllOccurs= []
for occ in occurrences: 
    hit = genome[occ: occ+len(p)]
    mismatch = 0
    for nt in range(len(x)):
        if hit[nt] != p[nt]:
            mismatch += 1
    mismatchAllOccurs.append(mismatch)
print(mismatchAllOccurs)

[0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 2, 2, 1, 1, 1, 1, 2, 2, 2, 0, 0, 1, 1, 0, 0, 1, 1, 2]


### Question 5
Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?
(Don't consider reverse complements.)

**Hint:** You should be able to use the boyer_moore function (or the slower naive function) to double-check your answer.

In [132]:
print('k-mer indexing method returns %d hits' % totalHits)

k-mer indexing method returns 90 hits


<div class="alert alert-block alert-success" style="text-align:center">Double-check with Boyer-Moore matching </div>

In [133]:
# split pattern into substrings of length 8
partitions,_ = splitArray(p, 8)
hits = []
# for each substring run boyer-moore matching
for st in partitions:
    p_bm = BoyerMoore(st, alphabet='ACGT')
    occurences,_,_ = boyer_moore_with_counts(st, p_bm, genome) 
    hits.extend(occurences)
print('Boyer-Moore method returns %d hits' % len(hits))

Boyer-Moore method returns 90 hits


<h3 style="border-bottom: 1px solid">k-mer indexing method with subsequence</h3>
Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings.  We'll consider subsequences involving every N characters.  For example, if we split ATATAT into two substring partitions, we would get partitions ATA (the first half) and TAT (second half).  But if we split ATATAT into two  subsequences  by taking every other character, we would get AAA (first, third and fifth characters) and TTT (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as 111222, and splitting into two subsequences of every other character could be represented as 121212

In [134]:
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """

    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq

    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

**Problem:** Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

In [135]:
''' function: use k-mer index data to find matches '''
def k_mer_match_Subseq2mm(p, t, index):
    k = index.k
    ival = index.ival
    
    occurrences = []
    totalHits = 0
    
    #for each partition of p
    for leftOffsetTarget in range(ival):
        target = p[leftOffsetTarget:]
        indexHitsForPartition = index.query(target)
            
        #if match found
        if len(indexHitsForPartition)>0:
            
            '''count index hits'''
            totalHits += len(indexHitsForPartition)
                        
            #for every index hit
            for i in indexHitsForPartition:
                
                hitIndexOffset = i-leftOffsetTarget
                # if hit is not already added
                if occurrences.count(hitIndexOffset)==0:              
                    '''count mismatch at index hit offset: only 2 mismatch is allowed'''
                    mismatch = 0 
                    # For each nucleotide of p
                    for nt in range(len(p)):
                        # if mismatch at nt
                        if t[hitIndexOffset+nt] != p[nt]:
                            mismatch += 1

                    #if #mismatch <= 2 on all of p, append hit
                    if mismatch <= 2:
                        occurrences.append(hitIndexOffset)
        
    return occurrences, totalHits

<div class="alert alert-block alert-success" style="text-align:center">k-mer with subsequence Test 1 </div>

In [136]:
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)
occurrences, num_index_hits = k_mer_match_Subseq2mm(p, t, subseq_ind)
print(occurrences)
print(num_index_hits)

[0, 14]
6


<div class="alert alert-block alert-success" style="text-align:center">k-mer with subsequence Test 2 </div>

In [137]:
# Load King John by William Shakespeare
!wget http://www.gutenberg.org/ebooks/1110.txt.utf-8

--2022-12-09 19:51:56--  http://www.gutenberg.org/ebooks/1110.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/ebooks/1110.txt.utf-8 [following]
--2022-12-09 19:51:58--  https://www.gutenberg.org/ebooks/1110.txt.utf-8
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/1110/pg1110.txt [following]
--2022-12-09 19:51:59--  https://www.gutenberg.org/cache/epub/1110/pg1110.txt
Reusing existing connection to www.gutenberg.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 145815 (142K) [text/plain]
Saving to: ‘1110.txt.utf-8.1’

1110.txt.utf-8.1    100%[==============

In [138]:
t = open('1110.txt.utf-8').read()
p = 'English measure backward'
subseq_ind = SubseqIndex(t, 8, 3)
occurrences, num_index_hits = k_mer_match_Subseq2mm(p, t, subseq_ind)
print(occurrences)
print(num_index_hits)

[132574]
3


### Question 6
When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

In [139]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"
subseq_ind = SubseqIndex(genome, 8, 3)
occurrences, num_index_hits = k_mer_match_Subseq2mm(p, genome, subseq_ind)
print('k-mer indexing method with subsequence returns %d hits' % num_index_hits)

k-mer indexing method with subsequence returns 79 hits


<div class="alert alert-block alert-success" style="text-align:center">Verify matches </div>

In [140]:
mismatchAllOccurs= []
for occ in occurrences: 
    hit = genome[occ: occ+len(p)]
    mismatch = 0
    for nt in range(len(x)):
        if hit[nt] != p[nt]:
            mismatch += 1
    mismatchAllOccurs.append(mismatch)
print(mismatchAllOccurs)

[0, 1, 1, 1, 0, 1, 0, 1, 2, 0, 1, 0, 2, 2, 1, 2, 1, 2, 1]
